In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

b = chat.predict("How many planets are there?")

b

('\n\nAs of 2021, there are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Pluto used to be considered a planet, but it was reclassified as a "dwarf planet" in 2006.',
 'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')